#### Modifying schema in custom fields

Custom field types can customise the schema generated for them using the `__modify_schema__` class method.

`__modify_schema__` can also take a `field` argument which will have type `Optional[ModelField]`. _pydantic_ will inspect the signature of `__modify_schema__` to determine whether the `field` argument should be included.

In [1]:
from typing import Any, Callable, Dict, Generator, Optional
from pydantic import BaseModel, Field
from pydantic.fields import ModelField

In [2]:
class RestrictedAlphabetStr(str):
    @classmethod
    def __get_validators__(cls) -> Generator[Callable, None, None]:
        yield cls.validate
    
    @classmethod
    def validate(cls, value: str, field: ModelField):
        alphabet = field.field_info.extra["alphabet"]
        if any(c not in alphabet for c in value):
            raise ValueError(f"{value!r} is not restricted to {alphabet!r}")
        return cls(value)
    
    @classmethod
    def __modify_schema__(
        cls, field_schema: Dict[str, Any], field: Optional[ModelField]
    ):
        if field:
            alphabet = field.field_info.extra["alphabet"]
            field_schema["examples"] = [c * 3 for c in alphabet]

In [3]:
class MyModel(BaseModel):
    value: RestrictedAlphabetStr = Field(alphabet="ABC")

In [4]:
print(MyModel.schema_json(indent=2))

{
  "title": "MyModel",
  "type": "object",
  "properties": {
    "value": {
      "title": "Value",
      "alphabet": "ABC",
      "examples": [
        "AAA",
        "BBB",
        "CCC"
      ],
      "type": "string"
    }
  },
  "required": [
    "value"
  ]
}


#### JSON Schema Types

Types, custom field types, and constraints (like `max_length`) are mapped to the corresponding spec formats in the following priority order (when there is an equivalent available):

1. JSON Schema Core
2. JSON Schema Validation
3. OpenAPI Data Types
4. The standard `format` JSON field is used to define pydantic extensions for more complex `string` sub-types.

The field schema mapping from Python / pydantic to JSON Schema is done as follows:

<table style="width:100%"><thead><tr><th>Python type</th><th>JSON Schema Type</th><th>Additional JSON Schema</th><th>Defined in</th></tr></thead><tbody><tr><td><code>None</code></td><td><code>null</code></td><td></td><td>JSON Schema Core</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>Same for <code>type(None)</code> or <code>Literal[None]</code></em></td></tr><tr><td><code>bool</code></td><td><code>boolean</code></td><td></td><td>JSON Schema Core</td></tr><tr><td><code>str</code></td><td><code>string</code></td><td></td><td>JSON Schema Core</td></tr><tr><td><code>float</code></td><td><code>number</code></td><td></td><td>JSON Schema Core</td></tr><tr><td><code>int</code></td><td><code>integer</code></td><td></td><td>JSON Schema Validation</td></tr><tr><td><code>dict</code></td><td><code>object</code></td><td></td><td>JSON Schema Core</td></tr><tr><td><code>list</code></td><td><code>array</code></td><td><code>{"items": {}}</code></td><td>JSON Schema Core</td></tr><tr><td><code>tuple</code></td><td><code>array</code></td><td><code>{"items": {}}</code></td><td>JSON Schema Core</td></tr><tr><td><code>set</code></td><td><code>array</code></td><td><code>{"items": {}, "uniqueItems": true}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>frozenset</code></td><td><code>array</code></td><td><code>{"items": {}, "uniqueItems": true}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>List[str]</code></td><td><code>array</code></td><td><code>{"items": {"type": "string"}}</code></td><td>JSON Schema Validation</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>And equivalently for any other sub type, e.g. <code>List[int]</code>.</em></td></tr><tr><td><code>Tuple[str, ...]</code></td><td><code>array</code></td><td><code>{"items": {"type": "string"}}</code></td><td>JSON Schema Validation</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>And equivalently for any other sub type, e.g. <code>Tuple[int, ...]</code>.</em></td></tr><tr><td><code>Tuple[str, int]</code></td><td><code>array</code></td><td><code>{"items": [{"type": "string"}, {"type": "integer"}], "minItems": 2, "maxItems": 2}</code></td><td>JSON Schema Validation</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>And equivalently for any other set of subtypes. Note: If using schemas for OpenAPI, you shouldn'tuse this declaration, as it would not be valid in OpenAPI (although it is valid in JSONSchema).</em></td></tr><tr><td><code>Dict[str, int]</code></td><td><code>object</code></td><td><code>{"additionalProperties": {"type": "integer"}}</code></td><td>JSON Schema Validation</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>And equivalently for any other subfields for dicts. Have in mind that although you can use othertypes as keys for dicts with Pydantic, only strings are valid keys for JSON, and so, only str isvalid as JSON Schema key types.</em></td></tr><tr><td><code>Union[str, int]</code></td><td><code>anyOf</code></td><td><code>{"anyOf": [{"type": "string"}, {"type": "integer"}]}</code></td><td>JSON Schema Validation</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>And equivalently for any other subfields for unions.</em></td></tr><tr><td><code>Enum</code></td><td><code>enum</code></td><td><code>{"enum": [...]}</code></td><td>JSON Schema Validation</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>All the literal values in the enum are included in the definition.</em></td></tr><tr><td><code>SecretStr</code></td><td><code>string</code></td><td><code>{"writeOnly": true}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>SecretBytes</code></td><td><code>string</code></td><td><code>{"writeOnly": true}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>EmailStr</code></td><td><code>string</code></td><td><code>{"format": "email"}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>NameEmail</code></td><td><code>string</code></td><td><code>{"format": "name-email"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td><code>AnyUrl</code></td><td><code>string</code></td><td><code>{"format": "uri"}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>Pattern</code></td><td><code>string</code></td><td><code>{"format": "regex"}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>bytes</code></td><td><code>string</code></td><td><code>{"format": "binary"}</code></td><td>OpenAPI</td></tr><tr><td><code>Decimal</code></td><td><code>number</code></td><td></td><td>JSON Schema Core</td></tr><tr><td><code>UUID1</code></td><td><code>string</code></td><td><code>{"format": "uuid1"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td><code>UUID3</code></td><td><code>string</code></td><td><code>{"format": "uuid3"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td><code>UUID4</code></td><td><code>string</code></td><td><code>{"format": "uuid4"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td><code>UUID5</code></td><td><code>string</code></td><td><code>{"format": "uuid5"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td><code>UUID</code></td><td><code>string</code></td><td><code>{"format": "uuid"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>Suggested in OpenAPI.</em></td></tr><tr><td><code>FilePath</code></td><td><code>string</code></td><td><code>{"format": "file-path"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td><code>DirectoryPath</code></td><td><code>string</code></td><td><code>{"format": "directory-path"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td><code>Path</code></td><td><code>string</code></td><td><code>{"format": "path"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td><code>datetime</code></td><td><code>string</code></td><td><code>{"format": "date-time"}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>date</code></td><td><code>string</code></td><td><code>{"format": "date"}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>time</code></td><td><code>string</code></td><td><code>{"format": "time"}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>timedelta</code></td><td><code>number</code></td><td><code>{"format": "time-delta"}</code></td><td>Difference in seconds (a <code>float</code>), with Pydantic standard "format" extension</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>Suggested in JSON Schema repository's issues by maintainer.</em></td></tr><tr><td><code>Json</code></td><td><code>string</code></td><td><code>{"format": "json-string"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td><code>IPv4Address</code></td><td><code>string</code></td><td><code>{"format": "ipv4"}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>IPv6Address</code></td><td><code>string</code></td><td><code>{"format": "ipv6"}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>IPvAnyAddress</code></td><td><code>string</code></td><td><code>{"format": "ipvanyaddress"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>IPv4 or IPv6 address as used in <code>ipaddress</code> module</em></td></tr><tr><td><code>IPv4Interface</code></td><td><code>string</code></td><td><code>{"format": "ipv4interface"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>IPv4 interface as used in <code>ipaddress</code> module</em></td></tr><tr><td><code>IPv6Interface</code></td><td><code>string</code></td><td><code>{"format": "ipv6interface"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>IPv6 interface as used in <code>ipaddress</code> module</em></td></tr><tr><td><code>IPvAnyInterface</code></td><td><code>string</code></td><td><code>{"format": "ipvanyinterface"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>IPv4 or IPv6 interface as used in <code>ipaddress</code> module</em></td></tr><tr><td><code>IPv4Network</code></td><td><code>string</code></td><td><code>{"format": "ipv4network"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>IPv4 network as used in <code>ipaddress</code> module</em></td></tr><tr><td><code>IPv6Network</code></td><td><code>string</code></td><td><code>{"format": "ipv6network"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>IPv6 network as used in <code>ipaddress</code> module</em></td></tr><tr><td><code>IPvAnyNetwork</code></td><td><code>string</code></td><td><code>{"format": "ipvanynetwork"}</code></td><td>Pydantic standard "format" extension</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>IPv4 or IPv6 network as used in <code>ipaddress</code> module</em></td></tr><tr><td><code>StrictBool</code></td><td><code>boolean</code></td><td></td><td>JSON Schema Core</td></tr><tr><td><code>StrictStr</code></td><td><code>string</code></td><td></td><td>JSON Schema Core</td></tr><tr><td><code>ConstrainedStr</code></td><td><code>string</code></td><td></td><td>JSON Schema Core</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>If the type has values declared for the constraints, they are included as validations. See themapping for <code>constr</code> below.</em></td></tr><tr><td><code>constr(regex='^text$', min_length=2, max_length=10)</code></td><td><code>string</code></td><td><code>{"pattern": "^text$", "minLength": 2, "maxLength": 10}</code></td><td>JSON Schema Validation</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>Any argument not passed to the function (not defined) will not be included in the schema.</em></td></tr><tr><td><code>ConstrainedInt</code></td><td><code>integer</code></td><td></td><td>JSON Schema Core</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>If the type has values declared for the constraints, they are included as validations. See themapping for <code>conint</code> below.</em></td></tr><tr><td><code>conint(gt=1, ge=2, lt=6, le=5, multiple_of=2)</code></td><td><code>integer</code></td><td><code>{"maximum": 5, "exclusiveMaximum": 6, "minimum": 2, "exclusiveMinimum": 1, "multipleOf": 2}</code></td><td></td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>Any argument not passed to the function (not defined) will not be included in the schema.</em></td></tr><tr><td><code>PositiveInt</code></td><td><code>integer</code></td><td><code>{"exclusiveMinimum": 0}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>NegativeInt</code></td><td><code>integer</code></td><td><code>{"exclusiveMaximum": 0}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>NonNegativeInt</code></td><td><code>integer</code></td><td><code>{"minimum": 0}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>NonPositiveInt</code></td><td><code>integer</code></td><td><code>{"maximum": 0}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>ConstrainedFloat</code></td><td><code>number</code></td><td></td><td>JSON Schema Core</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>If the type has values declared for the constraints, they are included as validations. See themapping for <code>confloat</code> below.</em></td></tr><tr><td><code>confloat(gt=1, ge=2, lt=6, le=5, multiple_of=2)</code></td><td><code>number</code></td><td><code>{"maximum": 5, "exclusiveMaximum": 6, "minimum": 2, "exclusiveMinimum": 1, "multipleOf": 2}</code></td><td>JSON Schema Validation</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>Any argument not passed to the function (not defined) will not be included in the schema.</em></td></tr><tr><td><code>PositiveFloat</code></td><td><code>number</code></td><td><code>{"exclusiveMinimum": 0}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>NegativeFloat</code></td><td><code>number</code></td><td><code>{"exclusiveMaximum": 0}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>NonNegativeFloat</code></td><td><code>number</code></td><td><code>{"minimum": 0}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>NonPositiveFloat</code></td><td><code>number</code></td><td><code>{"maximum": 0}</code></td><td>JSON Schema Validation</td></tr><tr><td><code>ConstrainedDecimal</code></td><td><code>number</code></td><td></td><td>JSON Schema Core</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>If the type has values declared for the constraints, they are included as validations. See themapping for <code>condecimal</code> below.</em></td></tr><tr><td><code>condecimal(gt=1, ge=2, lt=6, le=5, multiple_of=2)</code></td><td><code>number</code></td><td><code>{"maximum": 5, "exclusiveMaximum": 6, "minimum": 2, "exclusiveMinimum": 1, "multipleOf": 2}</code></td><td>JSON Schema Validation</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>Any argument not passed to the function (not defined) will not be included in the schema.</em></td></tr><tr><td><code>BaseModel</code></td><td><code>object</code></td><td></td><td>JSON Schema Core</td></tr><tr><td colspan="4" style="border-top: none; padding-top: 0"><em>All the properties defined will be defined with standard JSON Schema, including submodels.</em></td></tr><tr><td><code>Color</code></td><td><code>string</code></td><td><code>{"format": "color"}</code></td><td>Pydantic standard "format" extension</td></tr></tbody></table>